## Random Search

In [ ]:
from cornac.data import Reader
from cornac.eval_methods import RatioSplit

# Load MovieLens 100K dataset
reader = Reader()
ml_data = reader.read(fpath='./datasets/ml-100k/u.data', fmt='UIRT', sep='\t')

# Define the evaluation metrics
eval_method = RatioSplit(
    data=ml_data,
    test_size=0.2,
    rating_threshold=0.0,
    exclude_unknowns=True,
    verbose=True,
    seed=42
)

In [ ]:
import random

# Rango de valores
k_range = [10, 20, 30, 40, 50]
lr_range = [0.001, 0.005, 0.01, 0.02]
reg_range = [0.001, 0.01, 0.05, 0.1]

# Cantidad de combinaciones a generar
num_trials = 10

# Elegir 10 combinaciones aleatorias (con repetición)
random_combinations = [
    (
        random.choice(k_range),
        random.choice(lr_range),
        random.choice(reg_range)
    )
    for _ in range(num_trials)
]

In [ ]:
import pandas as pd
from cornac.models import MF
from cornac.metrics import RMSE, MAE, Precision, NDCG


resultados_random = []

for k, lr, reg in random_combinations:
    print(f"\nEntrenando MF con k={k}, lr={lr}, reg={reg}")

    model = MF(
        k=k,
        learning_rate=lr,
        lambda_reg=reg,
        max_iter=50,
        verbose=False
    )

    model.fit(eval_method.train_set)

    metrics = eval_method.evaluate(
        model,
        metrics=[RMSE(), MAE(), Precision(k=5), NDCG(k=5)],
        user_based=True
    )

    res = dict(metrics[0].metric_avg_results)
    res.update({"k": k, "lr": lr, "reg": reg})
    resultados_random.append(res)


df_random = pd.DataFrame(resultados_random)
df_random = df_random.sort_values(by="RMSE")

print(df_random)

**Tabla de resultados de Random Search de hiperparámetros sobre MF**

| Índice |  k  |   lr   |  reg   |   RMSE       |   MAE        | Precision@5      | NDCG@5      | Train (s)  | Test (s) |
|--------|-----|--------|--------|--------------|--------------|------------------|-------------|------------|----------|
|   0    |  40 | 0.005  | 0.001  | 0.987754     | 0.799558     | **0.088723**     | **0.09533** | 0.040255   | 0.651297 |
|   1    |  30 | 0.001  | 0.001  | 0.910321     | 0.749434     | 0.068298         | 0.06378     | 0.036518   | 0.810599 |
|   2    |  10 | 0.010  | 0.050  | 0.925274     | 0.754462     | 0.039149         | 0.03803     | 0.041386   | 0.692893 |
|   3    |  40 | 0.020  | 0.001  | 1.175490     | 0.948993     | 0.050000         | 0.05052     | 0.055942   | 0.726719 |
|   4    |  40 | 0.010  | 0.050  | 0.926032     | 0.755954     | 0.058936         | 0.06151     | 0.072233   | 1.167263 |
|   5    |  10 | 0.010  | 0.010  | 0.995569     | 0.808141     | 0.039787         | 0.03835     | 0.022823   | 0.807756 |
|   6    |  20 | 0.010  | 0.010  | 1.033805     | 0.836087     | 0.052766         | 0.05332     | 0.051659   | 0.679237 |
|   7    |  10 | 0.020  | 0.100  | **0.905964** | **0.743172** | 0.014468         | 0.01270     | 0.023100   | -1.67645 |
|   8    |  20 | 0.010  | 0.001  | 1.069231     | 0.859641     | 0.051277         | 0.05051     | 0.082295   | 0.781150 |
|   9    |  40 | 0.020  | 0.010  | 1.086976     | 0.877900     | 0.055532         | 0.06012     | 0.050360   | 0.703733 |


**Mejor precisión numérica (RMSE más bajo)**

* Índice 7 (`k=10`, `lr=0.0020`, `reg=0.100`)
* Mejor RMSE
* Precision\@5 y NDCG\@5 muy bajo (1,4% y 1,3%)

Este modelo predice mejor el rating real, pero recomienda mal. No es ideal para sistemas donde importa el Top-N.


**Mejor ranking (Precision\@5 y NDCG\@5)**

  * Índice 0 (`k=40`, `lr=0.005`, `reg=0.001`)
  * Precision\@5 = 0.0887
  * NDCG\@5 = 0.09533
  * RMSE moerado: 0.9877 

Este modelo ofrece las mejores recomendaciones ordenadas para el usuario.

**Buen balance entre precisión y relevancia**

* Índice 1 (`k=30`, `lr=0.001`, `reg=0.001`)
* RMSE = 0.9103, MAE = 0.7494
* Precision@5 = 0.0683, NDCG@5 = 0.0638

Este modelo logra un buen equilibrio: errores bajos y recomendaciones razonablemente acertadas. Muy útil si buscás estabilidad general.

**Modelos que conviene evitar**

* Índices 3, 8 y 9: RMSE > 1.06, precisión y NDCG bajos.
* Esto puede deberse a:
  * Tasa de aprendizaje demasiado alta (lr=0.02)
  * k muy alto con reg baja → sobreajuste o inestabilidad.

Estos modelos probablemente no generalizan bien y ofrecen rankings pobres.


**Variables evaluadas**

| Hiperparámetro | Significado                                   |
| -------------- | --------------------------------------------- |
| `k`            | Dimensión de los vectores latentes (factores) |
| `lr`           | Tasa de aprendizaje                           |
| `reg`          | Término de regularización (L2)                |

**Tendencias al aumentar `k` (número de factores latentes)**

| Observación                                                    | Conclusión clave                                                      |
| -------------------------------------------------------------- | --------------------------------------------------------------------- |
| `k=10` o `k=20` tienden a ser más estables                     | Generan buen equilibrio entre precisión y relevancia                  |
| `k=40` o `k=50` reduce RMSE pero puede dañar `Precision@5`     | Más capacidad para aprender, pero también mayor riesgo de sobreajuste |
| `k=30` fue uno de los mejores balances                         | Buen rendimiento general con bajo error y buen ranking                |

**Conclusión**: Aumentar `k` mejora precisión numérica (RMSE) hasta cierto punto, pero puede disminuir la calidad de las recomendaciones si no se regula bien.

**Tendencias al variar `lr` (learning rate)**

| Observación                                                                 | Conclusión clave                                        |
| --------------------------------------------------------------------------- | ------------------------------------------------------- |
| `lr=0.001` generó algunos de los mejores modelos de ranking (`Precision@5`) | Aprendizaje más lento pero más fino, evita oscilaciones |
| `lr=0.005` balanceó RMSE y ranking en varios casos                          | Buena tasa media, rápida pero aún estable               |
| `lr=0.02` empeoró RMSE y ranking en casi todos los casos                    | Tasa alta → aprendizaje inestable o salto de mínimos    |

**Conclusión**: Tasas bajas (0.001–0.005) son más confiables. `lr=0.02` suele ser demasiado agresivo para MF.

**Tendencias al modificar `reg` (regularización L2)**

| Observación                                                 | Conclusión clave                                                           |
| ----------------------------------------------------------- | -------------------------------------------------------------------------- |
| `reg=0.001` rindió bien en ranking, pero no siempre en RMSE | Baja penalización permite aprendizaje libre, pero riesgo de overfitting    |
| `reg=0.1` bajó el RMSE pero redujo `Precision@5`            | Penaliza fuerte → buen ajuste general, pero limita flexibilidad del modelo |
| `reg=0.01` fue el mejor punto medio en Grid Search          | Resultados equilibrados                                                    |

**Conclusión**:

* Mayor regularización → mejor RMSE
* Menor regularización → mejor Precision\@5

---
**Conclusión general**

| Hiperparámetro | Aumentar...                             | Disminuir...                                           |
| -------------- | --------------------------------------- | ------------------------------------------------------ |
| `k`            | Mejora RMSE, reduce diversidad en Top-N | Reduce capacidad, pero mejora ranking si bien regulado |
| `lr`           | Puede provocar errores grandes          | Mejora estabilidad y ranking                           |
| `reg`          | Reduce overfitting, mejora RMSE         | Aumenta capacidad de ranking (con riesgo)              |


**Para un buen balance entre RMSE y Precision\@5**:

* `k` ≈ 30
* `lr` ≈ 0.001–0.005
* `reg` ≈ 0.001–0.01

Ideal al buscar tanto buenas predicciones como listas Top-N relevantes.

In [ ]:
import matplotlib.pyplot as plt

# Asegurarse de tener una copia limpia del DataFrame de random search
df_random = pd.DataFrame(resultados_random)

# Crear columna de configuración como string
df_random['config'] = df_random.apply(
    lambda row: f"k={row['k']},lr={row['lr']},reg={row['reg']}", axis=1
)

# Ordenar por RMSE para visualizar mejor
df_random = df_random.sort_values(by='RMSE')

# Crear gráfico
fig, ax1 = plt.subplots(figsize=(12, 6))
ax2 = ax1.twinx()

# RMSE en eje izquierdo (rojo)
ax1.plot(df_random['config'], df_random['RMSE'], 'o-', color='tab:red', label='RMSE')
ax1.set_ylabel('RMSE', color='tab:red')
ax1.tick_params(axis='y', labelcolor='tab:red')

# Precision@5 en eje derecho (azul)
ax2.plot(df_random['config'], df_random['Precision@5'], 's--', color='tab:blue', label='Precision@5')
ax2.set_ylabel('Precision@5', color='tab:blue')
ax2.tick_params(axis='y', labelcolor='tab:blue')

# Títulos y ajustes visuales
plt.title('Comparación de RMSE y Precision@5 - Random Search MF')
ax1.set_xticks(range(len(df_random)))
ax1.set_xticklabels(df_random['config'], rotation=45, ha='right')
plt.tight_layout()
plt.grid(True)
plt.show()
